In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score

In [2]:
import opendatasets as od

In [3]:
dataset= 'https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset?select=Fake.csv'

In [4]:
od.download(dataset)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 41.0M/41.0M [00:12<00:00, 3.54MB/s]


In [5]:
df = pd.read_csv('train.csv')
conversion_dict = {0: 'Real', 1: 'Fake'}
df['label'] = df['label'].replace(conversion_dict)
df.label.value_counts()

Fake    10413
Real    10387
Name: label, dtype: int64

In [6]:
df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Fake
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,Real
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Fake
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Fake
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Fake


In [7]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df.fillna("missing",inplace = True)

In [9]:
x_train,x_test,y_train,y_test=train_test_split(df['text'], df['label'], test_size=0.25, random_state=7, shuffle=True)
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.75)

In [10]:
vec_train=tfidf_vectorizer.fit_transform(x_train.values.astype("U"))
vec_test=tfidf_vectorizer.transform(x_test.values.astype("U"))

In [12]:
pac=MultinomialNB()
pac.fit(vec_train,y_train)

MultinomialNB()

In [13]:
y_pred = pac.predict(vec_test)
score = accuracy_score(y_test, y_pred)
print(f"PAC Accuracy: {round(score*100,2)}%")

PAC Accuracy: 85.1%


In [14]:
X = tfidf_vectorizer.transform(df['text'].values.astype('U'))

In [15]:
scores = cross_val_score(pac, X, df['label'].values, cv=5)
print(f"K Fold Accuracy: {round(scores.mean()*100,2)}%")

K Fold Accuracy: 86.89%


In [16]:
df_true = pd.read_csv('True.csv')
df_true['label'] = 'Real'
df_true_rep = [df_true['text'][i].replace('WASHINGTON (Reuters) - ', '').replace('LONDON (Reuters) - ', '').replace('(Reuters) - ', '') for i in range(len(df_true['text']))]
df_true['text'] = df_true_rep
df_fake = pd.read_csv('Fake.csv')
df_fake['label'] = 'Fake'
df_final = pd.concat([df_true, df_fake])
df_final = df_final.drop(['subject', 'date'], axis=1)
df_fake

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",Fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",Fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",Fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",Fake


In [17]:
def findlabel(newtext):
    vec_newtest = tfidf_vectorizer.transform([newtext])
    y_pred1 = pac.predict(vec_newtest)
    return y_pred1[0]

In [18]:
findlabel((df_true['text'][0]))

'Real'

In [19]:
findlabel((df_fake['text'][1]))

'Real'

In [20]:
sum([1 if findlabel((df_true['text'][i])) == 'Real' else 0 for i in range(len(df_true['text']))]) / df_true['text'].size

0.9498062286968296

In [21]:
sum([1 if findlabel((df_fake['text'][i])) == 'Fake' else 0 for i in range(len(df_fake['text']))]) / df_fake['text'].size

0.24632681742685575

In [22]:
import joblib

In [23]:
joblib.dump(pac, 'model.sav')

['model.sav']

In [24]:
joblib.dump(tfidf_vectorizer, 'vectorizer.sav')

['vectorizer.sav']

In [25]:
#findlabel((df_fake['text'][0]))

'Real'